## СТАРТОВЫЙ МОДУЛЬ
---
ДАННЫЙ МОДУЛЬ ПРЕДНАЗНАЧЕН ДЛЯ ИНИЦИИРОВАНИЯ МИКРОСХЕМЫ IMU, ЗАПУСКА ЦИКЛА СЧИТЫВАНИЯ ПОКАЗАНИЙ СЕНСОРОВ, РАСЧЕТА УГЛОВ ЭЙЛЕРА ПО ВСЕМ ДОСТУПНЫМ КАНАЛАМ.

### Блок импорта библиотек необходимых для работы модуля

In [ ]:
# Подключить библиотеки для мат обработки данных и генерации случайных данных
import numpy as np
import random
import math

# Подключить библиотеки для работы с датой/временем
from datetime import datetime
from time import sleep

# Подключить библиотеки для асинхронной работы
import asyncio
# Подключить библиотеки для работы с базой данных redis
import redis
# Подключить библиотеки для работы с JSON
import json
# Подключить библиотеки для работы с завершающими функциями Jupyter Lab
import atexit

# ------------------------------------------------------------------------
# Подключить пользовательские модули
# ------------------------------------------------------------------------
import sys
sys.path.append('/home/project/3D-position-2023/py/lib')

# Подключить модуль для работы с данными IMU сенсоров
from py.lib.lib_calculate_angles_rev01_v01 import CalculateAngles
# Работа с интервалами в стиле setInterval JavaScript
from py.lib.lib_setinterval_rev01_v01 import SetInterval
# Подключить модуль для работы с базой данных redis
from py.lib.lib_work_redis_rev01_v01 import *

/usr/local/lib/python3.9/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Блок настроек среды исполнения Jupyter lab

In [2]:
# Инструкция требующая повторно перезагружать пользовательские функции после изменения их кода
%load_ext autoreload
%autoreload 2

### Блок считывания данных с сенсоров: Акселерометра, Гироскопа, Магнитометра

In [ ]:
# Подготовить параметры для инстанцирования IMU объекта и асинхронного считывания и пересчета данных
CONNECT_DB      = ConnectDB() # сохранить подключение к БД
LIMIT_DATA_ARR  =  50 # задать ограничение на размер массива данных сырых данных, углов Эйлера и т.д.
LIMIT_WIN_ARR   =  4 # задать размер окна усреднения сырых данных от IMU
OPTION_IMU      = {
    'GyroFullScale'  : 1000,
    'GyroSampleRate' : 50,
    'GyroLowPass'    : 3,
    'AccFullScale'   : 2,
    'AccSampleRate'  : 50,
    'AccLowPass'     : 3,
    'MagSampleRate'  : 50,
    'Isr'            : 137
}

ASYNC_IMU_INTERVAL  = 0.03 # интервал опроса IMU

In [ ]:
# Инстанцировать объект ICM20948 IMU сенсора
IMU = CalculateAngles(CONNECT_DB, LIMIT_DATA_ARR, LIMIT_WIN_ARR)

In [ ]:
#def cleanup_imu():
#    IMU.ICM20948.powerOff() # завершающий код для корректного выключения IMU
#    print("Завершающая функция вызвана!")
## Регистрируем функцию cleanup_imu для вызова при завершении работы программы
#atexit.register(cleanup_imu)

# Код опроса IMU в асинхронном блоке
async def AsyncUpdateIMU():
    AsyncFunc = SetInterval( ASYNC_IMU_INTERVAL, IMU.UpdateAll )
    AsyncFunc.start()

await AsyncUpdateIMU()